In [ ]:
import re, sys, copy, numpy as np
from math import sqrt
from aoclib import list2string
from collections import defaultdict
from itertools import combinations
from operator import add, mul

lines = open('21.txt').read().strip()

program = [int(x) for x in lines.split(",")]

In [ ]:
def icc(program, inputs = None): # intcode computer
    memory = program.copy()
    for j in range(10000 - len(memory)):
        memory.append(0)
                      
    output = []
    alt_par_modes = False # parameter modes other than position mode are enabled / disabled by this boolean
    relative_base = 0

    i = 0
    while True:
        if alt_par_modes:
            alt_par_modes = False
            instruction_header = str(memory[i]).rjust(5, "0")
            opcode = int(instruction_header[-2:])
            par_modes = [int(x) for x in instruction_header[-3::-1]]
        else:
            opcode = memory[i]
            par_modes = [0, 0, 0]

        if opcode == 99:
            break
            
        idx1 = i + 1 if par_modes[0] == 1 else memory[i + 1]
        idx2 = i + 2 if par_modes[1] == 1 else memory[i + 2]
        idx3 = i + 3 if par_modes[2] == 1 else memory[i + 3]
        if par_modes[0] == 2:
            idx1 += relative_base
        if par_modes[1] == 2:
            idx2 += relative_base
        if par_modes[2] == 2:
            idx3 += relative_base
        
        match opcode:
            case 1 | 2:
                memory[idx3] = memory[idx1] + memory[idx2] if opcode == 1 else memory[idx1] * memory[idx2]
                i += 4
            case 3:
                if inputs:
                    input_ = int(inputs.pop(0))
                else:
                    input_ = int(input('Please provide some input: '))
                memory[idx1] = input_
                i += 2
            case 4:
                output.append(memory[idx1])
                i += 2
            case 5:
                i = memory[idx2] if memory[idx1] != 0 else i + 3
            case 6:
                i = memory[idx2] if memory[idx1] == 0 else i + 3
            case 7:
                memory[idx3] = 1 if memory[idx1] < memory[idx2] else 0
                i += 4
            case 8:
                memory[idx3] = 1 if memory[idx1] == memory[idx2] else 0
                i += 4
            case 9:
                relative_base += memory[idx1]
                i += 2
            case _:
                alt_par_modes = True
                continue
    return output
